In [ ]:
import pennylane as qml
import numpy as np


def construct_oracle_circuit(secret_string):
    """Returns a quantum function (not a QNode) that implements the oracle.

    The oracle should perform the following operation:
        |x>|y> -> |x>|y \oplus f(x, s)>,

    where f(x, s) is the sum of the bitwise product of x and s modulo 2:
        f(x, s) = (x[0] * s[0] + x[1] * s[1] + ...) % 2

    Args:
        secret_string (str): The secret bit string.

    Returns:
        function: A function that applies the sequence of quantum gates
        that will be implemented by the oracle.
    """

    def oracle_circuit():
        for i in range(5):
            if (secret_string[i] == "1"):
                qml.CNOT(wires=[i, 5]) # wire 5 is initialized to 1
        
    return oracle_circuit

def hadamard_everything():
    for i in range(6):
        qml.Hadamard(wires=i)
    
def find_secret_string(secret_string):
    """Find a secret bit string s by implementing the Bernstein-Vazirani algorithm.

    You will need to use construct_oracle_circuit to construct the oracle
    outside the QNode, and then use that oracle in your QNode (this is to make
    it feel more like the string is a secret, as it allows the oracle to be
    called without any additional arguments). Feel free to add additional
    functions to add more layers of abstraction to the algorithm.

    Args:
        secret_string (str): A secret, 5-bit string.

    Returns:
        qml.QNode: A QNode that implements Bernstein-Vazirani. The output of the
        QNode should be a single sample of 5 bits that matches the secret string.
    """
    oracle_circuit = construct_oracle_circuit(secret_string)
    
    dev = qml.device("default.qubit", wires=6, shots=1)

    @qml.qnode(dev)
    def bernstein_vazirani():
        """The Bernstein-Vazirani algorithm."""
        qml.PauliX(wires=[5]) # initialize last qubit to 1

        hadamard_everything()
        oracle_circuit()
        hadamard_everything()
        
        return qml.sample(wires=[0, 1, 2, 3, 4])

    return bernstein_vazirani